# Parameter Sweeps and Batch Processing

Power system studies frequently require running many simulations with systematically varied parameters. You might want to understand how load levels affect voltage stability, how different fault locations impact system response, or how controller tuning parameters influence dynamic behavior. Rather than manually running each scenario one at a time, ANDES provides several approaches for batch processing that can dramatically reduce the time required for comprehensive studies.

This tutorial covers three approaches to batch simulation: file-based parallel processing for large studies, in-memory loops for smaller studies, and the `pool` option for intermediate cases where you want both parallelism and programmatic access to results.

:::{note}
**Prerequisites:** This tutorial assumes familiarity with Python fundamentals including loops, dictionaries, and NumPy arrays. Complete {doc}`02-first-simulation` through {doc}`07-eigenvalue-analysis` before proceeding.
:::

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup

In [1]:
%matplotlib inline

import andes
import numpy as np
from matplotlib import pyplot as plt
import os
import shutil

andes.config_logger(stream_level=30)  # Reduce logging verbosity for batch runs

## File-Based Batch Processing

For large parametric studies involving hundreds or thousands of scenarios, the most efficient approach is to generate separate case files and then run them in parallel using the ANDES command-line interface. This approach leverages multi-core processors effectively and allows you to checkpoint progress by checking which output files exist.

### Generating Case Files

The workflow begins by loading a base case, modifying parameters programmatically, and saving each variation to a new file. In this example, we create three cases with load levels varying from 100% to 120% of the base value.

In [2]:
# Create output directory for the case files
os.makedirs('batch_cases', exist_ok=True)

# Load base case
kundur = andes.get_case('kundur/kundur_full.xlsx')
ss = andes.load(kundur)

In [3]:
# Get the base load value
p0_base = ss.PQ.get('p0', 'PQ_0')
print(f"Base load: {p0_base:.2f} MW")

# Create cases with load varying from 100% to 120% of base
N_CASES = 3
p0_values = np.linspace(p0_base, 1.2 * p0_base, N_CASES)

for value in p0_values:
    ss.PQ.alter('p0', 'PQ_0', value)
    file_name = f'batch_cases/kundur_p_{value:.2f}.xlsx'
    andes.io.dump(ss, 'xlsx', file_name, overwrite=True)
    print(f"Created: {file_name}")

Base load: 11.59 MW
Created: batch_cases/kundur_p_11.59.xlsx
Created: batch_cases/kundur_p_12.75.xlsx
Created: batch_cases/kundur_p_13.91.xlsx


### Running Cases in Parallel

Once the case files are generated, you can run all of them in parallel using the `andes run` command with wildcards. ANDES automatically detects the number of CPU cores and distributes the workload across them.

In [4]:
# Run all cases with time-domain simulation
!andes run batch_cases/*.xlsx -r tds


    _           _         | Version 1.9.3.post20+gd85520b62.d20260104
   /_\  _ _  __| |___ ___ | Python 3.11.14 on Darwin, 01/05/2026 02:45:47 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"
> Loaded generated Python code in "/Users/hcui7/.andes/pycode".
-> Processing 3 jobs on 8 CPUs.
Process 0 for "batch_cases/kundur_p_11.59.xlsx" started.
Process 1 for "batch_cases/kundur_p_12.75.xlsx" started.
Process 2 for "batch_cases/kundur_p_13.91.xlsx" started.


  0%|                                                    | 0/100 [00:00<?, ?%/s]

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
 14%|#####6                                  | 14.0/100 [00:00<00:00, 134.33%/s]

 28%|###########2                            | 28.0/100 [00:00<00:00, 130.05%/s]

 28%|###########2                            | 28.0/100 [00:00<00:00, 128.22%/s]

...

 70%|############################            | 70.0/100 [00:00<00:00, 130.06%/s]

 83%|#################################1      | 83.0/100 [00:00<00:00, 136.71%/s]

 84%|#################################6      | 84.0/100 [00:00<00:00, 130.83%/s]

100%|#######################################| 100.0/100 [00:00<00:00, 135.75%/s]


100%|#######################################| 100.0/100 [00:00<00:00, 134.45%/s]
100%|#######################################| 100.0/100 [00:00<00:00, 132.67%/s]
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
Log saved to "/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/andes-wnccjo2_/andes.log".
-> Multiprocessing finished in 1.3578 seconds.
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn(

If you need to limit CPU usage (for example, to leave resources for other tasks on a shared workstation), use the `--ncpu` flag to specify the maximum number of parallel processes.

In [5]:
# Limit to 2 parallel processes
!andes run batch_cases/*.xlsx -r tds --ncpu 2


    _           _         | Version 1.9.3.post20+gd85520b62.d20260104
   /_\  _ _  __| |___ ___ | Python 3.11.14 on Darwin, 01/05/2026 02:45:49 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".
-> Processing 3 jobs on 2 CPUs.
Process 0 for "batch_cases/kundur_p_11.59.xlsx" started.
Process 1 for "batch_cases/kundur_p_12.75.xlsx" started.


  0%|                                                    | 0/100 [00:00<?, ?%/s]

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.


<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
 10%|####                                    | 10.0/100 [00:00<00:00, 193.41%/s]

...

 48%|###################2                    | 48.0/100 [00:00<00:00, 145.65%/s]

 63%|#########################2              | 63.0/100 [00:00<00:00, 144.22%/s]

 79%|###############################6        | 79.0/100 [00:00<00:00, 148.35%/s]

 96%|######################################4 | 96.0/100 [00:00<00:00, 153.76%/s]

100%|#######################################| 100.0/100 [00:00<00:00, 150.92%/s]
Log saved to "/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/andes-axg1vp0v/andes.log".
-> Multiprocessing finished in 2.4702 seconds.
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


### Returning System Objects for Post-Processing

When you want to analyze results programmatically after batch execution, use `pool=True` in the Python API. This runs cases in parallel and returns a list of System objects that you can then analyze or plot.

In [6]:
# Run all cases and return System objects
systems = andes.run('batch_cases/*.xlsx', routine='tds', pool=True, verbose=30)

print(f"Completed {len(systems)} simulations")
print(f"Type of each result: {type(systems[0]).__name__}")

Cases are processed in the following order:
"batch_cases/kundur_p_11.59.xlsx"
"batch_cases/kundur_p_12.75.xlsx"
"batch_cases/kundur_p_13.91.xlsx"


<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.

Log saved to "/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/andes-tk07k09h/andes.log".
-> Multiprocessing finished in 2.5295 seconds.
Completed 3 simulations
Type of each result: System


/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/hcui7/miniforge3/envs/aa/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


In [7]:
# Plot results from all cases side by side
fig, axes = plt.subplots(1, len(systems), figsize=(12, 4))

for i, sys in enumerate(systems):
    sys.TDS.plt.plot(sys.GENROU.omega, ax=axes[i],
                     title=f'Case {i+1}', latex=False, show=False)

plt.tight_layout()
plt.show()

/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/ipykernel_24889/3866593875.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## In-Memory Parameter Sweeps

For smaller studies where generating files would be unnecessary overhead, you can simply loop through parameter values in Python and accumulate results. This approach is particularly useful for quick exploratory analysis during model development or for sweeps involving only a few cases.

### Power Flow Parameter Sweep

The following example sweeps load levels and records the resulting bus voltage profiles. Since power flow converges quickly, this type of sweep can complete in seconds even for many parameter values.

In [8]:
# Load system
ss = andes.run(kundur, no_output=True, default_config=True, verbose=30)

# View current load parameters
ss.PQ.as_df(vin=True)

-> Single process finished in 0.1443 seconds.


,idx,u,name,bus,Vn,p0,q0,vmax,vmin,owner
uid,,,,,,,,,,
0,PQ_0,1.0,PQ_0,7,230.0,11.59,-0.735,1.1,0.9,1
1,PQ_1,1.0,PQ_1,8,230.0,15.75,-0.899,1.1,0.9,1


In [9]:
# Define parameter sweep range
n_samples = 5
pq0_values = np.linspace(10, 14, n_samples)  # MW range for PQ_0

# Storage for results: voltage at each bus for each parameter value
v_results = np.zeros((ss.Bus.n, n_samples))

# Run power flow for each load level
for i, p0 in enumerate(pq0_values):
    ss.PQ.alter('p0', 'PQ_0', p0)
    ss.PFlow.run()
    v_results[:, i] = ss.dae.y[ss.Bus.v.a]

print(f"Completed {n_samples} power flow calculations")

Completed 5 power flow calculations


In [10]:
# Visualize how voltage profiles change with load level
plt.figure(figsize=(10, 5))
for i in range(n_samples):
    plt.plot(v_results[:, i], 'o-', label=f'P0={pq0_values[i]:.1f} MW')

plt.xlabel('Bus Index')
plt.ylabel('Voltage [p.u.]')
plt.title('Voltage Profile vs Load Level')
plt.legend()
plt.xticks(range(ss.Bus.n), ss.Bus.name.v, rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/ipykernel_24889/196058281.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Batch Time-Domain Simulation

The same looping approach works for time-domain simulations, though each iteration takes longer. This example demonstrates running multiple contingency scenarios with different line trips. For each scenario, we create a fresh System object, add the appropriate disturbance, run the simulation, and store the result.

In [11]:
# Find available lines for contingency analysis
ss = andes.load(kundur, setup=False)
print(f"Available lines: {ss.Line.idx.v}")

# Select a subset for demonstration
lines_to_test = ss.Line.idx.v[:3]

Available lines: ['Line_0', 'Line_1', 'Line_2', 'Line_3', 'Line_4', 'Line_5', 'Line_6', 'Line_7', 'Line_8', 'Line_9', 'Line_10', 'Line_11', 'Line_12', 'Line_13', 'Line_14']


In [12]:
results = dict()

for line_idx in lines_to_test:
    # Load fresh system with setup=False to allow adding devices
    ss = andes.load(kundur, setup=False)

    # Add line trip at t=1.0s, restore at t=1.1s
    ss.add('Toggle', dict(model='Line', dev=line_idx, t=1.0))
    ss.add('Toggle', dict(model='Line', dev=line_idx, t=1.1))

    # Finalize system setup
    ss.setup()

    # Disable any existing Toggle from the test case
    ss.Toggle.alter('u', 1, 0.0)

    # Run simulation
    ss.PFlow.run()
    ss.TDS.config.tf = 5
    ss.TDS.config.no_tqdm = 1
    ss.TDS.run()

    results[line_idx] = ss
    print(f"Completed: {line_idx} trip")

<Toggle Toggle_2>: Line.Line_0 status changed to 0 at t=1.0 sec.
<Toggle Toggle_3>: Line.Line_0 status changed to 1 at t=1.1 sec.
Completed: Line_0 trip


<Toggle Toggle_2>: Line.Line_1 status changed to 0 at t=1.0 sec.
<Toggle Toggle_3>: Line.Line_1 status changed to 1 at t=1.1 sec.
Completed: Line_1 trip
<Toggle Toggle_2>: Line.Line_2 status changed to 0 at t=1.0 sec.
<Toggle Toggle_3>: Line.Line_2 status changed to 1 at t=1.1 sec.


Completed: Line_2 trip


In [13]:
# Compare generator response across contingencies
fig, axes = plt.subplots(1, len(results), figsize=(12, 4))

for ax, (line_idx, ss) in zip(axes, results.items()):
    ss.TDS.plt.plot(ss.GENROU.omega, ax=ax,
                    title=f'{line_idx} Trip', latex=False, show=False)

plt.tight_layout()
plt.show()

/var/folders/45/yt95k1qd4cj0wt64977wkb8w0000gr/T/ipykernel_24889/1233498955.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Choosing the Right Approach

The following table summarizes when to use each batch processing technique:

| Approach | Best For | Parallelism | Memory Usage |
|----------|----------|-------------|---------------|
| File-based CLI | Large studies (>100 cases), production runs | Multi-process | Low (one case at a time) |
| `pool=True` API | Medium studies, need results in memory | Multi-process | High (all results in memory) |
| Python loop | Small studies, rapid prototyping | Single-thread | Controlled (can discard results) |

For studies with more than about 10 cases, the file-based parallel approach is usually fastest because it fully utilizes all CPU cores. For smaller studies or when you are actively developing and testing, the simpler Python loop avoids the overhead of file I/O.

## Cleanup

In [14]:
shutil.rmtree('batch_cases', ignore_errors=True)
!andes misc -C

"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.npz" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.lst" removed.


## Next Steps

- {doc}`09-contingency-analysis` - Systematic N-1 contingency screening
- {doc}`10-dynamic-control` - Runtime parameter modifications
- {doc}`11-frequency-response` - Frequency response analysis